In [ ]:
%load_ext autoreload
%autoreload 2


import torch
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report, f1_score

from data_utils import HateDataset, get_results
from utils import initialize_seeds
from randomized_testing import randomized_test

In [ ]:
initialize_seeds()

In [ ]:
filename = "./data/targetData.pt"

In [ ]:
data = torch.load(filename)

## Before fine tuning

In [ ]:
model_names = ["davidson2017", "founta2018"]
all_preds = {}

In [ ]:
for name in model_names:
    model = BertForSequenceClassification.from_pretrained(f"./hatecheck-experiments/Models/BERT_{name}_weighted/Final")
    trainer = Trainer(
        model=model,         
        args=TrainingArguments(
            output_dir=(f"./hatecheck-experiments/Models/BERT_{name}_weighted/test"),
            per_device_eval_batch_size = 64)
    )
    for dataset in data[2]:
        print(f"Evaluating model pretrained on {name} on {dataset} test set")
        _, preds = get_results(trainer, data[2][dataset])
        all_preds[(name, dataset)] = preds

## After fine tuning

In [ ]:
splits = ["random", "unseen"]

In [ ]:
for name in model_names:
    for split in splits:
        if split == "random":
            model = BertForSequenceClassification.from_pretrained(f"./hatecheck-experiments/Models/BERT_{name}_hatecheck_weighted/Final")
        else:
            model = BertForSequenceClassification.from_pretrained(f"./hatecheck-experiments/Models/BERT_{name}_hatecheck_weighted_unseen/Final")
        trainer = Trainer(
            model=model,         
            args=TrainingArguments(
                output_dir=(f"./hatecheck-experiments/Models/test"),
                per_device_eval_batch_size = 64)
        )
        for dataset in data[2]:
            print(f"Evaluating model pretrained on {name} on {dataset} test set and finetuned on hatecheck with {split} split")
            _, preds = get_results(trainer, data[2][dataset])
            all_preds[(f"{name}-{split}", dataset)] = preds

## Data augmentation

In [ ]:
for name in ["davidson", "founta"]:
    model = BertForSequenceClassification.from_pretrained(f"./models/BERT_hateCheck+{name}_weighted/final")
    trainer = Trainer(
        model=model,         
        args=TrainingArguments(
            output_dir=(f"./hatecheck-experiments/Models/BERT_{name}_weighted/test"),
            per_device_eval_batch_size = 64)
    )
    for dataset in data[2]:
        print(f"Evaluating model pretrained on {name} on {dataset} test set")
        _, preds = get_results(trainer, data[2][dataset])
        all_preds[(f"hatecheck+{name}", dataset)] = preds

## Models that have never seen target data

In [ ]:
for split in splits:
        if split == "random":
            model = BertForSequenceClassification.from_pretrained(f"./models/BERT_hateCheck_weighted/final")
        else:
            model = BertForSequenceClassification.from_pretrained(f"./models/BERT_hateCheck_weighted_unseen/final")
        trainer = Trainer(
            model=model,         
            args=TrainingArguments(
                output_dir=(f"./hatecheck-experiments/Models/test"),
                per_device_eval_batch_size = 64)
        )
        for dataset in data[2]:
            print(f"Evaluating model that has never seen target data on {dataset} test set and finetuned on hatecheck with {split} split")
            _, preds = get_results(trainer, data[2][dataset])
            all_preds[(f"BertOnly_{split}", dataset)] = preds

In [ ]:
all_preds.keys()

In [ ]:
davidson_labels = data[2]["davidson2017"].labels
founta_labels = data[2]["founta2018"].labels

In [ ]:
def compare(model1, model2, labels, trials):
    preds1 = all_preds[model1]
    preds2 = all_preds[model2]
    randomized_test(preds1, preds2, labels, trials)

## Change after fine-tuning or mixed data trainin

In [ ]:
compare(('davidson2017', 'davidson2017'), ("davidson2017-random", "davidson2017"), davidson_labels, 10000)

In [ ]:
compare(('davidson2017', 'davidson2017'), ("hatecheck+davidson", "davidson2017"), davidson_labels, 10000)

In [ ]:
compare(('davidson2017', 'founta2018'), ("davidson2017-random", "founta2018"), founta_labels, 10000)

In [ ]:
compare(('davidson2017', 'founta2018'), ("hatecheck+davidson", "founta2018"), founta_labels, 10000)

In [ ]:
compare(('founta2018', 'founta2018'), ("founta2018-random", "founta2018"), founta_labels, 10000)

In [ ]:
compare(('founta2018', 'founta2018'), ("hatecheck+founta", "founta2018"), founta_labels, 10000)

In [ ]:
compare(('founta2018', 'davidson2017'), ("founta2018-random", "davidson2017"), davidson_labels, 10000)

In [ ]:
compare(('founta2018', 'davidson2017'), ("hatecheck+founta", "davidson2017"), davidson_labels, 10000)

In [ ]:
compare(('davidson2017-random', 'davidson2017'), ("davidson2017", "davidson2017"), davidson_labels, 10000)

In [ ]:
compare(('davidson2017-random', 'davidson2017'), ("BertOnly_random", "davidson2017"), davidson_labels, 10000)

In [ ]:
compare(('founta2018-random', 'founta2018'), ("BertOnly_random", "founta2018"), founta_labels, 10000)

## Fine-tuning vs mixed training

In [ ]:
compare(("davidson2017-random", "davidson2017"), ("hatecheck+davidson", "davidson2017"), davidson_labels, 10000)

In [ ]:
compare(("davidson2017-random", "founta2018"), ("hatecheck+davidson", "founta2018"), founta_labels, 10000)

In [ ]:
compare(("founta2018-random", "founta2018"), ("hatecheck+founta", "founta2018"), founta_labels, 10000)

In [ ]:
compare(("founta2018-random", "davidson2017"), ("hatecheck+founta", "davidson2017"), davidson_labels, 10000)